# Homework 3 - What is the best anime in the world?

## 1. Data collection

In [2]:
import requests as r
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import os
from datetime import datetime
import csv
import re
import nltk

### 1.1. Get the list of animes

In [2]:
# 19130 anime in total, 383 pages with 50 animes

# number of total pages
n_pages = 4 

with open("C:/Users/andre/Desktop/list_anime.txt", "w", encoding='utf-8') as file:
    
    for page in tqdm(range(0, n_pages)):
    
        # URL of the single page 
        url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
        
        # Get response object
        response = r.get(url)
        
        # Get html
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all links iterating through the document
        for tag in soup.find_all("tr", class_="ranking-list"):
            a_list = tag.find_all('a', class_="hoverinfo_trigger fl-l ml12 mr8" ,href=True)
            for a in a_list:
                link = a['href']
                file.write(str(link) + '\n')
                
file.close()

100%|██████████| 2/2 [00:01<00:00,  1.44it/s]


### 1.2. Crawl animes

In [3]:
file = open("C:/Users/andre/Desktop/prova_anime.txt", "r", encoding='utf-8')

counter_anime = 0
page = 0

for line in file:
    counter_anime += 1
    if (counter%50 == 1):
        page +=1
    response = r.get(line)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    try:
        os.mkdir("C:/Users/andre/Desktop/Anime/Page" + str(page))
    except OSError:
        pass
    
    with open("C:/Users/andre/Desktop/Anime/Page" + str(page) + "/article_" + str(counter_anime)+ ".html", "w", encoding='utf-8') as file:
        file.write(str(soup))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/andre/Desktop/prova_anime.txt'

In [ ]:
# Count anime links in file

file = open("C:/Users/andre/Desktop/list_anime.txt", "r", encoding='utf-8')
line_count = 0
for line in file:
    if line != "\n":
        line_count += 1
file.close()

print(line_count)

### 1.3 Parse downloaded pages

In [113]:
# Retrive information at page i of anime j
def retrive_information(i, j):
        with open("C:/Users/andre/Desktop/Anime/Page" + str(i) + "/anime" + str(j) + ".html", "r", encoding='utf-8') as file:
    
            # Read file and parse html code
            content_page = file.read()
            soup = BeautifulSoup(content_page, 'html.parser')
            
            # Retrive information 
            animeTitle = soup.find_all("h1", class_="title-name h1_bold_none")[0].find("strong").text
        
            animeType = soup.find_all('a', href = re.compile(r'type*'))[0].text.strip()
            
            animeNumEpisode = soup.find(string='Episodes:').next_element.replace("\n","").strip()
            
            animeNumMembers = soup.find(string='Members:').next_element.replace("\n","").replace(',','').replace('#', '').strip()
        
            animeScore = soup.find('span', {"itemprop":"ratingValue"}).text.strip()
            
            animeUsers = soup.find('span', {"itemprop":"ratingCount"}).text.strip()
            
            animeRank = soup.find(string='Ranked:').next_element.replace("\n","").replace('#','').strip()
            
            animePopularity = soup.find(string='Popularity:').next_element.replace("\n","").replace('#', '').strip()
            
            animeDescription = soup.find('p',{'itemprop':'description'}).text.replace('\n', '').replace('  ', '').strip()

    
            # Iterate in di with dates   
            dates_div = soup.find_all("div", class_="spaceit_pad")
            for i in range(0, len(dates_div)):
        
                # Dates
                if dates_div[i].find("span", string='Aired:') != None:
                    dates = dates_div[i].contents[2].strip().split("to")
                    try:
                        releaseDate = datetime.strptime(dates[0].strip(), '%b %d, %Y').date()
                    except:
                        releaseDate = None
                    try:
                        endDate = datetime.strptime(dates[1].strip(), '%b %d, %Y').date()
                    except:  
                        endDate = None
    
            # Anime Description
            animeDescription = soup.find("p", itemprop='description').text
    
            # Related Animes
            try:
                related_anime_table = soup.find("table", class_="anime_detail_related_anime")
                links_list = [link.text for link in related_anime_table.find_all("a")]
                animeRelated = list(set(links_list))
            except:
                animeRelated = None
    
            # Characters
            animeCharacters = soup.find_all("h3", class_="h3_characters_voice_actors")
            animeCharacters = [character.text for character in animeCharacters]
    
            # Voices
            animeVoices = soup.find_all("td", class_="va-t ar pl4 pr4")
            animeVoices = [actor.find("a").text for actor in animeVoices]
    
            a_no = []
            total = []
            result = []
            animeStaff = []
    
            # Staff 
            if animeCharacters == [] and animeVoices == []:
                div = soup.find_all("div", class_="detail-characters-list clearfix")[0]
                tds = div.find_all("td", class_=lambda x: x != 'ac borderClass' and x != 'ac' and x == 'borderClass')
    
                smalls = div.find_all("small")
    
                for td in tds:
                    a_no = set(list(td.find_all("a", class_="fw-n")))
                    total = set(list(td.find_all("a")))
                    if list(total.difference(a_no)) != []:
                        result.append(list(total.difference(a_no)))
    
                for (s, role) in zip(result, smalls):
                    s = str(s).strip("[").strip("]")
                    htmlTag = BeautifulSoup(s, "html.parser")
                    animeStaff.append([htmlTag.text, role.text])
                
            # There are no staff
            elif len(soup.find_all("div", class_="detail-characters-list clearfix")) < 2:
                animeStaff = []
                
            # There are one of animeCharacters or animeVoices
            else:
                div = soup.find_all("div", class_="detail-characters-list clearfix")[1]
                
                tds = div.find_all("td", class_=lambda x: x != 'ac borderClass' and x != 'ac' and x == 'borderClass')

                smalls = div.find_all("small")

                for td in tds:
                    a_no = set(list(td.find_all("a", class_="fw-n")))
                    total = set(list(td.find_all("a")))
                    if list(total.difference(a_no)) != []:
                        result.append(list(total.difference(a_no)))

                for (s, role) in zip(result, smalls):
                    s = str(s).strip("[").strip("]")
                    htmlTag = BeautifulSoup(s, "html.parser")
                    animeStaff.append([htmlTag.text, role.text])
            
        # Create list with overall information
        final_information = [animeTitle, animeType,animeNumEpisode,releaseDate,endDate,
                         animeNumMembers, animeScore,animeUsers,animeRank,animePopularity,
                         animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff]
        return final_information

In [114]:
retrive_information(1,30)

['Sen to Chihiro no Kamikakushi',
 'Movie',
 '1',
 datetime.date(2001, 7, 20),
 None,
 '1490571',
 '8.79',
 '1042818',
 '30',
 '43',
 "Stubborn, spoiled, and naïve, 10-year-old Chihiro Ogino is less than pleased when she and her parents discover an abandoned amusement park on the way to their new house. Cautiously venturing inside, she realizes that there is more to this place than meets the eye, as strange things begin to happen once dusk falls. Ghostly apparitions and food that turns her parents into pigs are just the start—Chihiro has unwittingly crossed over into the spirit world. Now trapped, she must summon the courage to live and work amongst spirits, with the help of the enigmatic Haku and the cast of unique characters she meets along the way.\n\n\nVivid and intriguing, Sen to Chihiro no Kamikakushi tells the story of Chihiro's journey through an unfamiliar world as she strives to save her parents and return home.\n\n\n[Written by MAL Rewrite]",
 None,
 ['Haku',
  'Ogino, Chihi

In [129]:
# Create .tsv file
n_pages = 2

# Collecting all Urls
with open("C:/Users/andre/Desktop/Anime/list_anime.txt", 'r', encoding='utf-8') as f_urls:
    urls = f_urls.readlines()
f_urls.close

# Create .tsv with only 3 important fields
with open("C:/Users/andre/Desktop/Anime/documents.tsv", 'w', encoding='utf-8') as f:
    tsv_writer = csv.writer(f, delimiter='\t')
            
    # Write haeders
    tsv_writer.writerow(['animeTitle','animeDescription','Url'])
            
f.close

# Iterating through pages and then animes
for i in range(1, n_pages):
    
    # Start and end computed with respect to the 
    start_iterate_anime = (i-1)*50+1
    end_iterate_anime = ((i-1)*50+1)+50
    for j in range(start_iterate_anime, end_iterate_anime):
        
        # Write for each anime a .tsv file
        with open("C:/Users/andre/Desktop/Anime/Page" + str(i) + "/anime_" + str(j)+ ".tsv", 'w', encoding='utf-8') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            
            # Write haeders
            tsv_writer.writerow(['animeTitle', 'animeType','animeNumEpisode','releaseDate','endDate',
                                 'animeNumMembers', 'animeScore','animeUsers','animeRank','animePopularity',
                                 'animeDescription','animeRelated','animeCharacters','animeVoices','animeStaff'])
            
            # Write information
            information = retrive_information(i, j) 
            tsv_writer.writerow(information)
        out_file.close
        
        lista = [information[0],information[10], urls[j]]
        
        # Append data to .tsv with only 3 important fields
        with open("C:/Users/andre/Desktop/Anime/documents.tsv", 'a', encoding='utf-8') as f:
            tsv_writer = csv.writer(f, delimiter='\t')
            
            # Write information 
            tsv_writer.writerow(lista)
            
        f.close


## 2. Search Engine

In [7]:
df = pd.read_csv("C:/Users/andre/Desktop/Anime/documents.tsv", sep='\t')
df

,animeTitle,animeDescription,Url
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/28977/Gintama°\r\n
1,Gintama°,"Gintoki, Shinpachi, and Kagura return as the f...",https://myanimelist.net/anime/38524/Shingeki_n...
2,Shingeki no Kyojin Season 3 Part 2,Seeking to restore humanity's diminishing hope...,https://myanimelist.net/anime/9253/Steins_Gate...
3,Steins;Gate,The self-proclaimed mad scientist Rintarou Oka...,https://myanimelist.net/anime/42938/Fruits_Bas...
4,Fruits Basket: The Final,"Hundreds of years ago, the Chinese Zodiac spir...",https://myanimelist.net/anime/9969/Gintama\r\n
5,Gintama',"After a one-year hiatus, Shinpachi Shimura ret...",https://myanimelist.net/anime/11061/Hunter_x_H...
6,Hunter x Hunter (2011),Hunter x Hunter is set in a world where Hunter...,https://myanimelist.net/anime/820/Ginga_Eiyuu_...
7,Ginga Eiyuu Densetsu,The 150-year-long stalemate between the two in...,https://myanimelist.net/anime/15417/Gintama__E...
8,Gintama': Enchousen,"While Gintoki Sakata was away, the Yorozuya fo...",https://myanimelist.net/anime/39486/Gintama__T...
9,Gintama: The Final,New Gintama movie.,https://myanimelist.net/anime/34096/Gintama\r\n


In [3]:
import nltk
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
nltk.download('stopwords')
nltk.download('punkt')
default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english') 
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [ ]:
def clean_text(text, ):

    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

    def remove_special_characters(text, characters=string.punctuation.replace('-', '')):
        tokens = tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(characters)))
        return ' '.join(filter(None, [pattern.sub('', t) for t in tokens]))

    def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

    def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

    text = text.strip(' ') # strip whitespaces
    text = text.lower() # lowercase
    text = stem_text(text) # stemming
    text = remove_special_characters(text) # remove punctuation and symbols
    text = remove_stopwords(text) # remove stopwords
    #text.strip(' ') # strip whitespaces again?

    return text


In [13]:
l = []
tokenizer = RegexpTokenizer(r"\w+")
ps = PorterStemmer()

def clean(text):
    tokens = tokenizer.tokenize(text.lower())
    for word in tokens:
        if word in default_stopwords:
            tokens.remove(word)
        l.append(ps.stem(word))
    t = l
    return ' '.join([str(elem) for elem in t])

In [34]:
df_descriptions = df['animeDescription']
df_descriptions = pd.DataFrame(df_descriptions)
df_descriptions

,animeDescription
0,After a horrific alchemy experiment goes wrong...
1,"Gintoki, Shinpachi, and Kagura return as the f..."
2,Seeking to restore humanity's diminishing hope...
3,The self-proclaimed mad scientist Rintarou Oka...
4,"Hundreds of years ago, the Chinese Zodiac spir..."
5,"After a one-year hiatus, Shinpachi Shimura ret..."
6,Hunter x Hunter is set in a world where Hunter...
7,The 150-year-long stalemate between the two in...
8,"While Gintoki Sakata was away, the Yorozuya fo..."
9,New Gintama movie.


In [33]:
df_descriptions = df_descriptions.applymap(lambda x : clean(x), na_action='ignore')
df_descriptions

,animeDescription
0,after horrif alchemi experi goe wrong in elric...
1,after horrif alchemi experi goe wrong in elric...
2,after horrif alchemi experi goe wrong in elric...
3,after horrif alchemi experi goe wrong in elric...
4,after horrif alchemi experi goe wrong in elric...
5,after horrif alchemi experi goe wrong in elric...
6,after horrif alchemi experi goe wrong in elric...
7,after horrif alchemi experi goe wrong in elric...
8,after horrif alchemi experi goe wrong in elric...
9,after horrif alchemi experi goe wrong in elric...
